In [1]:
import numpy as np

class Chromosome:
    
    def crossover(self, other):
        pass
    
    def mutate(self):
        pass
    
    def getFitness(self):
        pass
    
    def __str__(self):
        pass
    

class GeneticAlgorithm:
    
    def __init__(self, popSize, generations, cross, mutate):
        self.populationSize = popSize
        self.numGenerations = generations
        self.probC = cross
        self.probM = cross
        self.population = [None for i in range(0,popSize)]
        self.roulette_min = [0 for i in range(0,popSize)]
        self.roulette_max = [0 for i in range(0,popSize)]
        
    def buildInitialPopulation(self):
        pass
    
    def calculateRoulette(self):
        sum = 0
        for chromosome in self.population:
            sum = sum + chromosome.getFitness()
        
        self.roulette_min[0] = 0
        for i in range(0, self.populationSize):
            if i != 0:
                self.roulette_min[i] = self.roulette_max[i-1]
            self.roulette_max[i] = self.roulette_min[i] + self.population[i].getFitness() / sum
            
        #print(self.roulette_min)
        #print(self.roulette_max)

    def pickChromosome(self):
        spin = np.random.uniform()
        for i in range(0,self.populationSize):
            if spin > self.roulette_min[i] and spin <= self.roulette_max[i]:
                return i
        return self.populationSize-1

    def reproductionLoop(self):
        newPop = []
        
        for i in range(0, self.populationSize, 2):
            
            x = self.population[self.pickChromosome()]
            y = self.population[self.pickChromosome()]
            
            if (np.random.uniform() < self.probC):
                x, y = x.crossover(y)
                
            if (np.random.uniform() < self.probM):
                x = x.mutate()
                
            if (np.random.uniform() < self.probM):
                y = y.mutate()
            
            newPop.append(x)
            newPop.append(y)
            
        self.population = newPop    
    
    
    def printResults(self):
        
        best = self.population[0]
        fit = []
        
        for chromosome in self.population:
            fit.append(chromosome.getFitness())
            if chromosome.getFitness() > best.getFitness():
                best = chromosome
       
       # print("Population Fitness")
       # print(fit)
                
        print("Best State: " + str(best.getFitness()))
        #print(best)
        return best
    
    def runGA(self, target=0):
        
        best = None
        self.buildInitialPopulation()
        for i in range(0, self.numGenerations):
            
            self.calculateRoulette()
            
            self.reproductionLoop()
            
            best = self.printResults()
            if best.getFitness() >= target:
                print("Solution found at generation " + str(i))
                break
    
        print(best)
        return best
    
    
class QueenChromosome(Chromosome):
    
    def __init__(self, n, state=None):
        self.state = []
        self.numQueens = n

        if state == None:
            for i in range(0,n):
                self.state.append(np.random.randint(0,n))
            
        else:
            self.state = state[:]
        self.calculateFitness()
    
    def crossover(self, other): 
        point = np.random.randint(0,self.numQueens)
        x1 = self.state[0:point]
        x2 = self.state[point:]
        y1 = other.state[0:point]
        y2 = other.state[point:]
        
        child1 = QueenChromosome(self.numQueens, x1 + y2)
        child2 = QueenChromosome(self.numQueens, y1 + x2)
        
        return child1, child2
    
    def mutate(self):
        newState = self.state[:]
        point = np.random.randint(0,self.numQueens)
        newVal = self.state[point]
        while (newVal == self.state[point]):
            newVal = np.random.randint(0, self.numQueens)
        newState[point] = newVal
        return QueenChromosome(self.numQueens, newState)
        
    def calculateFitness(self):
    
        self.fitness = 0
        rowFitness  = 0
        
        for row in range(0,self.numQueens):
            rowFitness = self.numQueens - 1
            
            col = self.state[row]
            
            for i in range(0, self.numQueens):
                
                if i == row:
                    continue
                
                # Check for other rows with same column
                if (self.state[i] == col):
                    rowFitness = rowFitness - 1
                #diagonal 1
                if (self.state[i] == (col + (i-row))):
                    rowFitness = rowFitness - 1
                #diagonal 2
                if (self.state[i] == (col + (i-row))):
                    rowFitness = rowFitness - 1
        
            self.fitness = self.fitness + rowFitness
        self.fitness = self.fitness
    

    def getFitness(self):
        return self.fitness

    
    def __str__(self):
        """
        Converts the state into a string representation of an N x N chess board
        where N is defined by the desired number of queens.
        
        Queen represented by a "Q".  Empty represented by a "."
        
        @return string representation of the node's chess board state
        """
        boardStr = ""
        
        # Traverse row by row
        for i in range(0,self.numQueens):
            # Initialize row as a string list with "." for blank
            row = ["." for x in range(0,self.numQueens)]
            
            # If row has queen, update list at queen index with Q
            if self.state[i] != -1:
                row[self.state[i]] = "Q"
                
            # Convert list to a string and concatenate
            boardStr += " ".join(map(str, row)) + "\n"
        return boardStr
    
class QueensGA(GeneticAlgorithm):
    
    def __init__(self, popSize, generations, cross, mutate, numQueens=8):
        self.numQueens = numQueens
        super().__init__(popSize, generations, cross, mutate)
    
    def buildInitialPopulation(self):
        self.population = []
        for i in range(self.populationSize):
            self.population.append(QueenChromosome(self.numQueens))
            


In [2]:
q = QueensGA(50, 1000, 0.7, 0.1, 8)
q.runGA(target=56)

Best State: 52
Best State: 52
Best State: 54
Best State: 52
Best State: 52
Best State: 54
Best State: 54
Best State: 52
Best State: 52
Best State: 52
Best State: 54
Best State: 52
Best State: 52
Best State: 50
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 52
Best State: 50
Best State: 54
Best State: 52
Best State: 54
Best State: 52
Best State: 50
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 54
Best State: 52
Best State: 54
Best State: 52
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 52
Best State: 52
Best State: 52
Best State: 50
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 56
Solution found at generation 56
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .
. . . . . . Q .
. . . Q . . . .
. Q . . . . . .
Q

In [5]:
q = QueensGA(100, 1000, 0.5, 0.01, 16)
q.runGA(target=16*15)

Best State: 224
Best State: 228
Best State: 224
Best State: 226
Best State: 226
Best State: 224
Best State: 222
Best State: 218
Best State: 220
Best State: 220
Best State: 220
Best State: 226
Best State: 226
Best State: 226
Best State: 226
Best State: 228
Best State: 226
Best State: 226
Best State: 222
Best State: 228
Best State: 230
Best State: 228
Best State: 226
Best State: 224
Best State: 224
Best State: 224
Best State: 218
Best State: 220
Best State: 222
Best State: 222
Best State: 222
Best State: 224
Best State: 232
Best State: 222
Best State: 226
Best State: 224
Best State: 226
Best State: 224
Best State: 220
Best State: 224
Best State: 224
Best State: 226
Best State: 224
Best State: 230
Best State: 230
Best State: 230
Best State: 232
Best State: 230
Best State: 228
Best State: 228
Best State: 228
Best State: 222
Best State: 224
Best State: 226
Best State: 226
Best State: 228
Best State: 228
Best State: 228
Best State: 228
Best State: 226
Best State: 222
Best State: 230
Best Sta

Best State: 226
Best State: 226
Best State: 228
Best State: 226
Best State: 226
Best State: 228
Best State: 232
Best State: 228
Best State: 228
Best State: 230
Best State: 228
Best State: 230
Best State: 230
Best State: 228
Best State: 228
Best State: 228
Best State: 226
Best State: 224
Best State: 228
Best State: 228
Best State: 226
Best State: 226
Best State: 226
Best State: 226
Best State: 228
Best State: 226
Best State: 228
Best State: 230
Best State: 230
Best State: 230
Best State: 228
Best State: 228
Best State: 226
Best State: 226
Best State: 226
Best State: 226
Best State: 226
Best State: 232
Best State: 226
Best State: 226
Best State: 224
Best State: 226
Best State: 230
Best State: 226
Best State: 230
Best State: 230
Best State: 226
Best State: 226
Best State: 224
Best State: 224
Best State: 222
Best State: 226
Best State: 226
Best State: 226
Best State: 228
Best State: 230
Best State: 226
Best State: 226
Best State: 228
Best State: 228
Best State: 228
Best State: 228
Best Sta